<a href="https://colab.research.google.com/github/pratheeshkumar99/Neural-machine-translation/blob/main/NMT_eng_to_hindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
data = pd.read_csv('./drive/My Drive/Hindi_English_Truncated_Corpus .csv')

In [2]:
import tensorflow  as tf

#tf.enable_eager_execution()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import time
import string
import pandas as pd

# import plotly
# import plotly.plotly as py
# from plotly.offline import init_notebook_mode, iplot
# plotly.offline.init_notebook_mode(connected=True)
# import plotly.graph_objs as go

In [3]:
data.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory


import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import tensorflow as tf


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

# Any results you write to the current directory are saved as output.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [5]:
lines=pd.read_csv("./drive/My Drive/Hindi_English_Truncated_Corpus .csv",encoding='utf-8')

In [6]:
lines['source'].value_counts()

tides        50000
ted          39881
indic2012    37726
Name: source, dtype: int64

In [7]:
lines.head(20)

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what needs to be done.,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है ."
1,ted,"I'd like to tell you about one such child,","मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,"
2,indic2012,This percentage is even greater than the percentage in India.,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not paying attention.,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called Upanishad.,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
5,tides,"The then Governor of Kashmir resisted transfer , but was finally reduced to subjection with the aid of British .","कश्मीर के तत्कालीन गवर्नर ने इस हस्तांतरण का विरोध किया था , लेकिन अंग्रेजों की सहायता से उनकी आवाज दबा दी गयी ."
6,indic2012,In this lies the circumstances of people before you.,इसमें तुमसे पूर्व गुज़रे हुए लोगों के हालात हैं।
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
8,indic2012,“”Global Warming“” refer to warming caused in recent decades and probability of its continual presence and its indirect effect on human being.,ग्लोबल वॉर्मिंग से आशय हाल ही के दशकों में हुई वार्मिंग और इसके निरंतर बने रहने के अनुमान और इसके अप्रत्यक्ष रूप से मानव पर पड़ने वाले प्रभाव से है।
9,tides,You may want your child to go to a school that is not run by the LEA - a non-maintained special school or an independent school that can meet your child 's needs .,"हो सकता है कि आप चाहते हों कि आप का नऋर्नमेनटेन्ड ह्यबिना किसी समर्थन के हृ विशेष स्कूल , या किसी स्वतंत्र स्कूल में जाए , इजसके पास विशेष शैक्षणिक जऋऋरतों वाले बच्चों के प्रति सहूलियत हों . ."


In [8]:
pd.isnull(lines).sum()

source              0
english_sentence    2
hindi_sentence      0
dtype: int64

In [9]:
lines=lines[~pd.isnull(lines['english_sentence'])]

In [10]:
lines.drop_duplicates(inplace=True)
lines.shape

(124827, 3)

In [11]:
lines=lines.sample(n=124827,random_state=42)
lines.shape

(124827, 3)

In [12]:
# Lowercase all characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())

In [13]:
# Remove quotes
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [14]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [15]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))
print(lines['english_sentence'])
lines.shape


25520     islam is word from arabic and it full word is salamaits definition peace surrender                                                    
118633    everything is reliant on these computers working                                                                                      
113495    parliament does not control the government                                                                                            
29783     race equality new laws                                                                                                                
111804    the provision would not affect the power of parliament to make laws in respect of income from professions etc  lrb article  rrb       
                                                                       ...                                                                      
122524    the judge too moved  as if to rise  but remembering the judicial convention  kept sitting                               

(124827, 3)

In [16]:
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# # Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
print(lines['english_sentence'])


25520     islam is word from arabic and it full word is salamaits definition peace surrender                                                   
118633    everything is reliant on these computers working                                                                                     
113495    parliament does not control the government                                                                                           
29783     race equality new laws                                                                                                               
111804    the provision would not affect the power of parliament to make laws in respect of income from professions etc  lrb article  rrb      
                                                                       ...                                                                     
122524    the judge too moved  as if to rise  but remembering the judicial convention  kept sitting                                     

In [17]:
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))
#print(lines['english_sentence'])

In [18]:
# Add start and end tokens to target sequences
lines['english_sentence'] = lines['english_sentence'].apply(lambda x : '<start> '+ x + ' <end>')
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x : '<start> '+ x + ' <end>')

In [19]:
lines.head()

,source,english_sentence,hindi_sentence
25520,indic2012,<start> islam is word from arabic and it full word is salamaits definition peace surrender <end>,<start> इस्लाम शब्द अरबी भाषा का शब्द है जिसका मूल शब्द सल्लमा है जिस की दो परिभाषाएं हैं शान्ति आत्मसमर्पण। <end>
118633,ted,<start> everything is reliant on these computers working <end>,<start> इन कंप्यूटरों पर सब कुछ निर्भर है <end>
113495,tides,<start> parliament does not control the government <end>,<start> संसद का सरकार पपर नियंत्रण नपहीं रहता <end>
29783,tides,<start> race equality new laws <end>,<start> नये कानून नस्ली समानता <end>
111804,tides,<start> the provision would not affect the power of parliament to make laws in respect of income from professions etc lrb article rrb <end>,<start> व्यवसायों आदि से होने वाली आय के बारे में विधि बनाने की संसद की शक्ति पर उपबंध का प्रभाव नहीं पड़ेगा अनुच्छेद <end>


In [20]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [21]:
len(all_eng_words)

72688

In [22]:
len(all_hindi_words)

81977

In [23]:
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [24]:
lines[lines['length_eng_sentence']>20].shape
lines[lines['length_eng_sentence']>20].shape

(35299, 5)

In [25]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

In [27]:
lines.shape

(81245, 5)

In [31]:
lines.head()

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
118633,ted,<start> everything is reliant on these computers working <end>,<start> इन कंप्यूटरों पर सब कुछ निर्भर है <end>,9,9
113495,tides,<start> parliament does not control the government <end>,<start> संसद का सरकार पपर नियंत्रण नपहीं रहता <end>,8,9
29783,tides,<start> race equality new laws <end>,<start> नये कानून नस्ली समानता <end>,6,6
57202,ted,<start> there was lasagna there was casseroles <end>,<start> वहां लाजान्या था कैसेरोल थे <end>,8,7
107821,indic2012,<start> super power india source google writer vedpratap vedik <end>,<start> महाशक्ति भारत गूगल पुस्तक लेखक वेदप्रताप वैदिक <end>,10,9


In [72]:
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [73]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(57949, 66369)

In [74]:
eng = []
hindi = []
for line in lines['english_sentence']:
    eng.append(line)

for line in lines['hindi_sentence']:
    hindi.append(line)


In [75]:
data = zip(*(eng,hindi))
data = list(data)

In [76]:
# This class creates a word -> index mapping (e.g,. "dad" -> 5) and vice-versa 
# (e.g., 5 -> "dad") for each language,
class LanguageIndex():
    def __init__(self, lang):
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()

        self.create_index()

    def create_index(self):
        for phrase in self.lang:
            self.vocab.update(phrase.split(' '))

        self.vocab = sorted(self.vocab)

        self.word2idx['<pad>'] = 0
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1

        for word, index in self.word2idx.items():
            self.idx2word[index] = word

In [77]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [78]:
def load_dataset(pairs, num_examples):
    # pairs => already created cleaned input, output pairs

    # index language using the class defined above    
    inp_lang = LanguageIndex(en for en, ma in pairs)
    targ_lang = LanguageIndex(ma for en, ma in pairs)
    
    # Vectorize the input and target languages
    
    # English sentences
    input_tensor = [[inp_lang.word2idx[s] for s in en.split(' ')] for en, ma in pairs]
    
    # Marathi sentences
    target_tensor = [[targ_lang.word2idx[s] for s in ma.split(' ')] for en, ma in pairs]
    
    # Calculate max_length of input and output tensor
    # Here, we'll set those to the longest sentence in the dataset
    max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
    
    # Padding the input and output tensor to the maximum length
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
                                                                 maxlen=max_length_inp,
                                                                 padding='post')
    
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, 
                                                                  maxlen=max_length_tar, 
                                                                  padding='post')
    
    return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar

In [79]:
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(data, len(data))

In [80]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.1, random_state = 101)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(66694, 66694, 7411, 7411)

In [105]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 128
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)
print("English vocab_Size:",vocab_inp_size)
print("Marati vocab_Size:",vocab_tar_size)

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

English vocab_Size: 50382
Marati vocab_Size: 55818


In [106]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
        return tf.keras.layers.GRU(units, 
                                   return_sequences=True, 
                                   return_state=True, 
                                   recurrent_activation='sigmoid', 
                                   recurrent_initializer='glorot_uniform')

In [107]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [108]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [109]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE) #5679,256,1024,64
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE) #13647,256,1024,64

In [110]:
optimizer =tf.keras.optimizers.Adam()

def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

In [111]:
checkpoint_dir = './drive/My Drive/train_C/'
checkpoint_prefix = os.path.join(checkpoint_dir, "check_points")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [112]:
EPOCHS = 20
from tqdm import tqdm 

for epoch in tqdm(range(EPOCHS)):
    start = time.time()
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in (enumerate(dataset)):
        loss = 0
        #print(batch)
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(targ[:, t], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every epoch
    checkpoint.save(file_prefix = checkpoint_prefix)
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))







  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1 Batch 0 Loss 4.0944
Epoch 1 Batch 100 Loss 2.6995
Epoch 1 Batch 200 Loss 2.8925
Epoch 1 Batch 300 Loss 2.7752
Epoch 1 Batch 400 Loss 2.6138
Epoch 1 Batch 500 Loss 2.5567








  5%|▌         | 1/20 [14:07<4:28:27, 847.75s/it]

Epoch 1 Loss 2.7105
Time taken for 1 epoch 847.7484939098358 sec

Epoch 2 Batch 0 Loss 2.2925
Epoch 2 Batch 100 Loss 2.3194
Epoch 2 Batch 200 Loss 2.4368
Epoch 2 Batch 300 Loss 2.3721
Epoch 2 Batch 400 Loss 2.3213
Epoch 2 Batch 500 Loss 2.5575








 10%|█         | 2/20 [28:23<4:15:01, 850.10s/it]

...

Epoch 18 Loss 0.2942
Time taken for 1 epoch 882.5060198307037 sec

Epoch 19 Batch 0 Loss 0.2088
Epoch 19 Batch 100 Loss 0.2004
Epoch 19 Batch 200 Loss 0.1945
Epoch 19 Batch 300 Loss 0.2472
Epoch 19 Batch 400 Loss 0.1867
Epoch 19 Batch 500 Loss 0.2104








 95%|█████████▌| 19/20 [4:35:59<14:36, 876.83s/it]

Epoch 19 Loss 0.2222
Time taken for 1 epoch 884.1838736534119 sec

Epoch 20 Batch 0 Loss 0.1665
Epoch 20 Batch 100 Loss 0.1444
Epoch 20 Batch 200 Loss 0.1753
Epoch 20 Batch 300 Loss 0.1904
Epoch 20 Batch 400 Loss 0.2001
Epoch 20 Batch 500 Loss 0.2270








100%|██████████| 20/20 [4:50:37<00:00, 871.85s/it]

Epoch 20 Loss 0.1800
Time taken for 1 epoch 877.3177452087402 sec



In [113]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint('./drive/My Drive/train_C/'))

In [117]:
def evaluate(inputs, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    
    #attention_plot = np.zeros((max_length_targ, max_length_inp))
    sentence = ''
    for i in inputs[0]:
        if i == 0:
            break
        sentence = sentence + inp_lang.idx2word[i] + ' '  # GEtting out the actual sentence from the indexed representation
    sentence = sentence[:-1]    # removing the end token
    inputs = tf.convert_to_tensor(inputs) #converting the list into a tensor to fed into the model
    
    result = ''

    hidden = [tf.zeros((1, units))]  #initialising the hidden-states of the encoder
    enc_out, enc_hidden = encoder(inputs, hidden) #feeding the input and hidden states to the encoder

    dec_hidden = enc_hidden  #initialising the  decoder's hidden state as the encoder's output
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0) #hard-coding the first time step decoder input ad start token as expanding its dims so as to include batch size since the model accepts batch_size of imputs

    for t in range(max_length_targ):
      
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out) #The decoder return the ouput of that time step as-well as the hidden states(to be fed into the attention mechganism for the next time step) and attention weights
        
        # storing the attention weights to plot later on
        #attention_weights = tf.reshape(attention_weights, (-1, ))
        #attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.idx2word[predicted_id] + ' '
        #print(result)

        if targ_lang.idx2word[predicted_id] == '<end>': #stop the decoder loop if end token is emitted 
            return result, sentence
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence

In [120]:
def predict_random_val_sentence():
    actual_sent = ''
    k = np.random.randint(len(input_tensor_val))
    random_input = input_tensor_val[k]
    random_output = target_tensor_val[k]
    random_input = np.expand_dims(random_input,0)
    result, sentence = evaluate(random_input, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
    print(sentence)
    print(result)
    print('Input: {}'.format(sentence[8:-6]))
    print('Predicted translation: {}'.format(result[:-6]))
    for i in random_output:
        if i == 0:
            break
        actual_sent = actual_sent + targ_lang.idx2word[i] + ' '
    actual_sent = actual_sent[8:-7]
    print('Actual translation: {}'.format(actual_sent))
    #attention_plot = attention_plot[:len(result.split(' '))-2, 1:len(sentence.split(' '))-1]
    sentence, result = sentence.split(' '), result.split(' ')
    sentence = sentence[1:-1]
    result = result[:-2]
    
    # # use plotly to generate the heat map
    # trace = go.Heatmap(z = attention_plot, x = sentence, y = result, colorscale='Reds')
    # data=[trace]
    # iplot(data)

In [132]:
predict_random_val_sentence()

<start> pandava kunthi and madhuri are all in pandus <end>
तौरात और पाण्डु के मध्य में सभी बहुमूल्य पत्थरों का पूर्ण हैं। <end> 
Input: pandava kunthi and madhuri are all in pandus
Predicted translation: तौरात और पाण्डु के मध्य में सभी बहुमूल्य पत्थरों का पूर्ण हैं। 
Actual translation: पाण्डव पाण्डु की कुन्ती और माद्री से सन्ताने।
